# Threat Hunting for Gretel Processing
Hugging Face Dataset 



### SETUP

In [76]:
%pip install jupyter ipykernel
import os
#VIRTUALENVGOODNESS - YEAH BRAHHHHHH
ROOT_DIR = '../'

VIRTUALENV=True
#################
VENV_NAME="venv"
if VIRTUALENV == True:
    try:
        os.chdir(ROOT_DIR)
        print(os.path.curdir)
        !source $VENV_NAME/bin/activate
    except Exception as e:
        print(e)
        %pip3 install virtualenv
        !virtualenv $VENV_NAME
        !source $VENV_NAME/bin/activate
        %python3 -m ipykernel install  --user --name=



Note: you may need to restart the kernel to use updated packages.
.
zsh:source:1: no such file or directory: venv/bin/activate


In [77]:
%pip install -q torch
%pip install numpy
%pip install pandas
%pip install -q wandb #comet_ml
%pip install pyarrow python-dotenv datasets gretel-client




Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Constants


In [78]:
RUNS=1
PROJECT_NAME="Cognitive Synthesis"
DEBUG = True
LOG_LEVEL = DEBUG
PARALLEL = True


In [79]:
if RUNS == 0:
  try:
    %pprint
    DEFAULT_GH_PROJECT="synavatelabs_project"
    GH_USER = 'snyata' #this is your git user name
    NOTEBOOK_ERRORS = []
    RUNS+=1
    print("##### INITIAL SETUP COMPLETE ######")
  except Exception as e:
    print(e)
else:
  pass

In [80]:
import os
import random
from dotenv import load_dotenv
import numpy as np
import torch
from tqdm.auto import tqdm

# SET TO FALSE IF NOT ON GOOGLE COLAB
COLAB = False

# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)

# Device configuration
device = torch.device("mps" if COLAB == False & torch.backends.mps.is_available() else "cpu")
print(f"Environment Colab == {COLAB}\n\n Your device is {device}")
if COLAB == False:
    print("FOR GPU ON COLAB SET COLAB TO TRUE")

Environment Colab == False

 Your device is mps
FOR GPU ON COLAB SET COLAB TO TRUE


# Data PreProcessing
### Logging and safe execution decorator

In [81]:
import sys
import os
from typing import Callable, Any
from functools import wraps
import notebook_logging as logging

logger = logging.setup_logging()

# Setup basic configuration for logging
#logging.basicConfig(level='logging.{}, LOG_LEVEL', filename=f'./data/logs/{PROJECT_NAME}_safe_apply_errors.log',
#                    format='%(asctime)s:%(levelname)s:%(message)s')

def safe_apply(func: Callable[..., Any]) -> Callable[..., Any]:
    """
    A decorator to safely apply a function to a pandas DataFrame or Series column.

    Args:
        func: A function to be applied to a pandas DataFrame or Series column.

    Returns:
        The wrapped function with added exception handling.
    """
    @wraps(func)
    def wrapper(*args, **kwargs) -> Any:
        count_errors = 0
        try:
            # Assuming the first argument is always the DataFrame or Series
            result = func(*args, **kwargs)
            return result
        except Exception as e:
            count_errors += 1
            if count_errors > 2:
                logging.error(__name__)
                sys.exit()
            # Log the exception
            logging.error(f"Error applying function {func.__name__}: {e}")
            # Optionally, return None or a default value instead of raising
            return None

    return wrapper

In [82]:
#Hugging Face Data
from datasets import load_dataset
import pandas as pd
#load_dotenv()

DATASET = "Olec/cyber-threat-intelligence_v2"
ds = load_dataset(DATASET)
###Turns the DATSET into a set of tensors for PyTorch
#ds = ds.with_format("torch", device=device)




In [127]:
# Creating a pandas Dataframe from the training dataset


df = pd.DataFrame({
    "id": ds['train']["id"],
    "text": ds["train"]["text"],
    "entities": ds["train"]["entities"]
})
df['text'].head()
df.to_csv('/Users/nullzero/Documents/repos/github.com/synavate/products/Orion-Constellation/cognitive_synthesis/data/data/intelligence-data.csv')

In [123]:
def data_cleaning(df):
    if type(df) != pd.DataFrame:
        logging.error(f"#### ERROR: START OF CLEANING: {df} is type {type(df)} ####")
    else:
        logger.info(f"####START OF CLEANING: {df.columns} is type {type(df)} ####\n")
    logger.info(f"START SHAPE:  {df.shape}")
    # Remove N/A and duplicate text, id
    #df['text'] = df['text'].drop_duplicates(inplace=True)
    #df['id'] = df['id'].drop_duplicates(inplace=True)
    df_processed = df['text'].dropna()
    logger.info("######  SHAPE OF YOUR DATA IS  ######")
    logger.info(df_processed.shape)
    if type(df_processed) != pd.DataFrame:
        logger.error(f"####END OF PROCESSING: {df_processed} is type {type(df_processed)} ####")
    else:
        logger.info(f"####END OF PROCESSING: {df_processed} is type {type(df_processed)} ####")
    print("######  END OF DATA CLEANING  ######")
    df_processed.head()    
    return df_processed






## NLP Functions

In [85]:
%pip install -q tqdm IProgress
%pip install -q nltk
%pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [86]:
#Process Text NLP

import IProgress
from tqdm.notebook import tqdm
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
tqdm.pandas()

[nltk_data] Downloading package punkt to /Users/nullzero/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nullzero/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nullzero/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [87]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import pos_tag

In [88]:
#Label Variable For Operations on Entities
# Helpful to work out exactly where in the data structure the item is


#label_variable = df['entities'][0][2]['label']
#label_variable

# TEXT FUNCTIONS

In [89]:
from typing import List, Dict, Any 

In [107]:
@safe_apply
def reduce_text(df):
  print(df['text'])
  lemmatizer = WordNetLemmatizer()
  words = [lemmatizer.lemmatize(df['text']) for words in df['text'] if words not in stopwords.words('english')]
  words_list = ' '.join(words)
  return words_list

In [91]:
# TOKENIZE TEXT FUNCTION
@safe_apply
def tokenize_text(text):
  for word in text:  
    tokens = word_tokenize(word)
    tokens = [word for word in tokens if word.isalpha()]
  return ' '.join(tokens)


In [92]:
@safe_apply
def extract_features(entities):
    # Define the features you're interested in.
    features_of_interest = [
        "attack-pattern",
        "malware",
        "ip",
        "threat-actor",
        "campaign",
        "intrusion-set"
    ]
    
    # Initialize an empty list to hold labels (features) found in entities.
    features_found = []
    
    # Iterate through each entity dictionary in the entities list.
    
        # If the entity's label is one of the features of interest, append it to the list.
    if entities[0][0][2]['label'] in features_of_interest:
        features_found.append(entities[0][0][2]['label'])
        
    
    # Return the list with labels found in entities.
    return ', '.join(features_found)



In [93]:
# Isolate attack patterns
@safe_apply
def isolate_attack_patterns(entities):
    # Directly check 'label' in each entity dictionary
    attack_patterns = [entities[0][2]['label'] for entity in entities if entity == 'attack-pattern']
    attack_patterns = ', '.join(attack_patterns)
    logging.INFO("SUCCESS ATTACK LABELS")
    return attack_patterns



In [94]:


@safe_apply
def isolate_labels(data: pd.Series) -> pd.Series:
    # Pulls the labels out of the entities dictionaries into it's own column
    print(data)
    data["raw_labels"] = data['entities'][0][2]['labels']
    logging.INFO("SUCCESS LABELS ISOLATION")
    return data["raw_labels"]



In [95]:
#  ONE HOT ENCODING FUNCTION
from sklearn.preprocessing import OneHotEncoder

@safe_apply
def one_hot(data):
    one_hot = OneHotEncoder()
    labels_array = np.array(data).reshape(-1, 1)
    one_hot_encoded = one_hot.fit_transform(labels_array)
    logging.INFO("SUCCESS ONE HOT")
    return one_hot_encoded
    



In [75]:
#Classify
#@safe_apply
#def classify_text(text):
 #   return nltk.classify(text)

print ("### CLASSIFY NOT IMPLEMENTED ###")

### CLASSIFY NOT IMPLEMENTED ###


# Text Processing Procedure
PROGRESS_APPLY FOR TRACKING OF PANDAS EXECUTIOn - SAFE_APPLY WILL AUTOMATICALLY LOG INCLUDING ERRORS

### PARALLEL PROCESSING

### Executing Functions

In [109]:
'''

ALL EXECUTION OF PROGRESS APPLYING TEXT PROCESSING ARE IN THIS STEP FOR THE ORION LIBRARY

'''

###RUN DATA CLEANING #####
try:
    df_new = data_cleaning(df)
    df_new.head()
except Exception as e:
    logger.error(e)

2024-04-14 21:00:13,734:INFO:####START OF CLEANING: Index(['id', 'text', 'entities'], dtype='object') is type <class 'pandas.core.frame.DataFrame'> ####

2024-04-14 21:00:13,734:INFO:####START OF CLEANING: Index(['id', 'text', 'entities'], dtype='object') is type <class 'pandas.core.frame.DataFrame'> ####

2024-04-14 21:00:13,734:INFO:####START OF CLEANING: Index(['id', 'text', 'entities'], dtype='object') is type <class 'pandas.core.frame.DataFrame'> ####

2024-04-14 21:00:13,734:INFO:####START OF CLEANING: Index(['id', 'text', 'entities'], dtype='object') is type <class 'pandas.core.frame.DataFrame'> ####

2024-04-14 21:00:13,734:INFO:####START OF CLEANING: Index(['id', 'text', 'entities'], dtype='object') is type <class 'pandas.core.frame.DataFrame'> ####

2024-04-14 21:00:13,742:INFO:START SHAPE:  (332, 3)
2024-04-14 21:00:13,742:INFO:START SHAPE:  (332, 3)
2024-04-14 21:00:13,742:INFO:START SHAPE:  (332, 3)
2024-04-14 21:00:13,742:INFO:START SHAPE:  (332, 3)
2024-04-14 21:00:13,74

######  END OF DATA CLEANING  ######


In [112]:
from parallel_execution import parallel_execution_function

try:
    df_final = parallel_execution_function(df_new)
except Exception as e:
    logger.critical(e, exc_info=True)

/Users/nullzero/Documents/repos/github.com/synavate/products/Orion-Constellation/cognitive_synthesis/venv/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


       id  ...                                           entities
34   1708  ...  [{'end_offset': 75, 'id': 46883, 'label': 'mal...
35   4587  ...  [{'end_offset': 180, 'id': 50446, 'label': 'SO...
36    106  ...  [{'end_offset': 29, 'id': 45834, 'label': 'thr...
37   3410  ...  [{'end_offset': 11, 'id': 10289, 'label': 'mal...
38   3390  ...  [{'end_offset': 22, 'id': 10240, 'label': 'mal...
39   4265  ...  [{'end_offset': 250, 'id': 50067, 'label': 'th...
40   2399  ...  [{'end_offset': 32, 'id': 47472, 'label': 'TIM...
41  14408  ...  [{'end_offset': 84, 'id': 47166, 'label': 'att...
42    564  ...  [{'end_offset': 164, 'id': 1874, 'label': 'REG...
43   2313  ...  [{'end_offset': 68, 'id': 45456, 'label': 'vul...
44   3370  ...  [{'end_offset': 105, 'id': 10186, 'label': 'ma...
45   4459  ...  [{'end_offset': 115, 'id': 50241, 'label': 'ma...
46   4818  ...  [{'end_offset': 38, 'id': 50750, 'label': 'ide...
47    234  ...  [{'end_offset': 132, 'id': 45781, 'label': 'lo...
48   3830 

2024-04-14 21:07:23,442:CRITICAL:'Series' object has no attribute 'progress_apply'
multiprocessing.pool.RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
                    ^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/pool.py", line 48, in mapstar
    return list(map(*args))
           ^^^^^^^^^^^^^^^^
  File "/Users/nullzero/Documents/repos/github.com/synavate/products/Orion-Constellation/cognitive_synthesis/data/parallel_execution.py", line 52, in process_dataframe_chunk
    df_chunk['attack_patterns'] = df_chunk['entities'].progress_apply(isolate_attack_patterns)
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/nullzero/Documents/repos/github.com/synava

In [50]:


def run_all_apply(df):

    #Reduce text down using NLTK and apply from Pandas
    df["text"] = df["text"].progress_apply(reduce_text)
    print("Text reduced")

    # ATTACK PATTERNS
    df['attack_patterns'] = df['entities'].progress_apply(isolate_attack_patterns)
    print("Attack patterns done")

    # FEATURE EXTRACTION
    # Apply the function to each row's 'entities' column and store the result in a new column.
    df['features_found'] = df['entities'].progress_apply(extract_features)
    print("features extracted")

    # THREAT LABELS

    df['threat_labels'] = df['entities'].progress_apply(isolate_labels)
    print("Threat Labels extracted")

    # ONE HOT ENCODING
    #OneHot Encoding
    df['one_hot_labels']= df['threat_labels'].progress_apply(one_hot)
    print("One Hot Encoded")

    #Tokenize text
    df["tokens"] = df["text"].progress_apply(tokenize_text)
    print("Text tokenized")
    return df

import multiprocessing

PARALLEL= True
if PARALLEL:
    try:
        # Create a multiprocessing pool
        pool = multiprocessing.Pool()
        result = pool.map(run_all_apply, df.itertuples(index=False))
        
        # Assuming 'result' is a list of dictionaries, you can directly create a DataFrame
        df_result = pd.DataFrame(result)
        
        print("PARALLEL DONE - RESULT DF CREATED")
        
        # Close the multiprocessing pool
        pool.close()
        # Wait for all worker processes to finish
        pool.join()
        df_result.head()
        
    except Exception as e:
        print(e)
        logging.error(e)








Can't pickle <class 'pandas.core.frame.Pandas'>: attribute lookup Pandas on pandas.core.frame failed


AttributeError: module 'notebook_logging' has no attribute 'error'

In [ ]:
result_df.describe()

In [ ]:
result_df.shape



In [ ]:
result_df.head()

In [ ]:
### GCP Storage SOURCE BUCKET ###
%env SOURCE_BUCKET=example-state-backend


In [ ]:
import os
from ..gcp.gcp_upload import upload_blob
from datetime import datetime
# Turn processed dataframe to csv 
csv_output = df.tocsv(f"./processed/gretel-threat-intelligence-{datetime.now()}.csv")
%export DATA_FILE= csv_output
# Upload blob
upload_blob("gretel-threat-intelligence", os.getenv("DATA_FILE"), "gretel-threat-intelligence.csv")


In [ ]:
!gcloud storage cp $DATA_FILE gs://$SOURCE_BUCKET